In [ ]:
for r in range(2):
    time_start = time.time()

    accuracies = []

    print(f"START: {sub_lst}, {r+1}th subject")

    X_tr, Y_tr, X_te, Y_te = step1(sub_lst, train_init_rep, test_rep, Feature_idx)

    _, _, for_proto_X, for_proto_y = get_data(X_tr, Y_tr, X_te, Y_te, K_shot, split=True)


    input_shape = X_tr.shape[1:]

    model = build_model(num_classes, input_shape)
    model.compile(loss=lambda y_true, y_pred: custom_loss(y_true, y_pred, alpha), metrics=['accuracy'])



    train_embeddings = model.predict(for_proto_X, verbose=0)
    train_labels = np.argmax(for_proto_y, axis=1)
    prototypes = calculate_prototypes(train_embeddings, train_labels, num_classes)

    for meta_iter in range(meta_iters):
        frac_done = meta_iter / meta_iters
        cur_meta_step_size = (1 - frac_done) * meta_step_size
        old_vars = model.get_weights()

        #train_dataset = get_data(X_tr, Y_tr, X_te, Y_te, K_shot, split=False)
        train_dataset, test_data, for_proto_X, for_proto_y = get_data(X_tr, Y_tr, X_te, Y_te, K_shot, split=True)

        # Inner loop - episode
        #model.fit(mini_dataset, epochs=30, verbose=0)  # Train on the mini-batch
        result = model.fit(train_dataset, epochs=100, validation_data=test_data, callbacks=[MemoryCallback()], verbose=0)  # Outer loop training
        val_acc = np.max(result.history['val_accuracy'])
        accuracies.append(val_acc)
        print(f'Subject {r+1}  -  Iter {meta_iter}/{meta_iters} - {val_acc},  max so far: {np.max(accuracies)}')


        # Apply meta-update using the new model weights
        # 여기서 예전 가중치들이랑 최근꺼랑 업데이트
        new_vars = model.get_weights()
        for var in range(len(new_vars)):  # SGD for meta gradient
            new_vars[var] = old_vars[var] + ((new_vars[var] - old_vars[var]) * cur_meta_step_size)


        # meta 업데이트를 한 가중치들을 모델에다가 넣음
        model.set_weights(new_vars)  # After the meta-learning step, reload the newly-trained weights into the model.
    time_end = time.time()
    print(f"Total training time: {time_end - time_start:.2f} seconds")